In [ ]:
NLP Practical 3

In [ ]:
import numpy as np
import os
from random import shuffle
import re
import sklearn as sk
import matplotlib.pyplot as plt
import random

In [ ]:
import urllib.request
import zipfile
import lxml.etree

In [ ]:
import tensorflow as tf

In [ ]:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
    
raw_text = doc.xpath('//content/text()')
raw_label = doc.xpath('//head/keywords/text()')

del doc

In [ ]:
talk_sentences = []
talknum = len(raw_text)

for i in range(talknum):
    temp = re.sub(r'\([^)]*\)', '', raw_text[i])
    temp = re.sub(r'\n', '', raw_text[i])
    temp = temp.split('.')
    talk_sentences.append(temp)

In [ ]:
print(talk_sentences[0][1])

In [ ]:
talk_sentence_word = []

for talk in talk_sentences:
    temp = []
    for sent in talk:
        tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower()).split()
        temp.append(tokens)#
    talk_sentence_word.append(temp)

In [ ]:
talk_sentence_word[0][1]

In [ ]:
freq = {}

for talk in talk_sentence_word:
    for sent in talk:
        for word in sent:
            if word in freq:
                freq[word] += 1
            else:
                freq[word] = 1

In [ ]:
from collections import Counter

counts_ted_top1000 = []
words_top_ted = []

for word, count in Counter(freq).most_common(1000):
    counts_ted_top1000.append(count)
    words_top_ted.append(word)

In [ ]:
talk_sentence_word_nostop = talk_sentence_word

for stop in words_top_ted[:200]:
    for talk in talk_sentence_word_nostop:
        for sent in talk:
            for word in sent:
                if word == stop:
                    sent.remove(word)

In [ ]:
talk_sentence_word_nostop[0][1]

In [ ]:
np.shape(talk_sentence_word_nostop)

In [ ]:
input_label = []

for i in range(len(raw_label)):
    temp = re.sub(r' ', '', raw_label[i])
    input_label.append(temp.split(','))
    
#print(input_label_prepro)

In [ ]:
labels_binary = []

for i in range(len(input_label)):
    temp = np.zeros(3)
    if 'technology' in '~'.join(input_label[i]):
        temp[0] = 1
    if 'entertainment' in '~'.join(input_label[i]):
        temp[1] = 1
    if 'design' in '~'.join(input_label[i]):
        temp[2] = 1
    labels_binary.append(temp)
        
print(input_label[9])
print(labels_binary[:10])

In [ ]:
temp = list(zip(talk_sentence_word_nostop, labels_binary))
random.shuffle(temp)
talk_sentence_word_nostop_shuffle, labels_binary_shuffle = zip(*temp)

In [60]:
labels_onehot = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary[i][0]*4+labels_binary[i][1]*2+labels_binary[i][2]*1] = 1
    labels_onehot.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [61]:
labels_onehot[:10]

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])]

In [ ]:
labels_onehot_shuffle = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary_shuffle[i][0]*4+labels_binary_shuffle[i][1]*2+labels_binary_shuffle[i][2]*1] = 1
    labels_onehot_shuffle.append(temp)

In [ ]:
train_data = talk_sentence_word_nostop[:1835]
test_data = talk_sentence_word_nostop[1835:]

train_labels = labels_binary[:1835]
test_labels = labels_binary[1835:]

train_labels_onehot = labels_onehot[:1835]
test_labels_onehot = labels_onehot[1835:]

In [ ]:
train_data_shuffle = talk_sentence_word_nostop_shuffle[:1835]
test_data_shuffle = talk_sentence_word_nostop_shuffle[1835:]

train_labels_shuffle = labels_binary_shuffle[:1835]
test_labels_shuffle = labels_binary_shuffle[1835:]

train_labels_onehot_shuffle = labels_onehot_shuffle[:1835]
test_labels_onehot_shuffle = labels_onehot_shuffle[1835:]

In [ ]:
print(np.shape(talk_sentence_word_nostop))
print(np.shape(labels_binary))
print(np.shape(labels_onehot))
print(np.shape(train_data))
print(np.shape(train_labels))
print(np.shape(train_labels_onehot))
print(np.shape(test_data))
print(np.shape(test_labels))
print(np.shape(test_labels_onehot))

In [ ]:
print(np.shape(talk_sentence_word_nostop_shuffle))
print(np.shape(labels_binary_shuffle))
print(np.shape(labels_onehot_shuffle))
print(np.shape(train_data_shuffle))
print(np.shape(train_labels_shuffle))
print(np.shape(train_labels_onehot_shuffle))
print(np.shape(test_data_shuffle))
print(np.shape(test_labels_shuffle))
print(np.shape(test_labels_onehot_shuffle))

In [ ]:
check = np.asarray(train_labels_onehot)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot)
print(check.sum(0) / check.sum())

In [ ]:
check = np.asarray(train_labels_onehot_shuffle)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot_shuffle)
print(check.sum(0) / check.sum())

In [ ]:
train_lenghts = []

for talk in train_data:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    train_lenghts.append(N)

In [ ]:
train_lenghts[:10]

In [ ]:
test_lenghts = []

for talk in test_data:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    test_lenghts.append(N)

In [ ]:
test_lenghts[:10]

In [ ]:
train_talk_word = []

for talk in train_data:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    train_talk_word.append(temp)
    
print(len(train_talk_word[0]))

In [ ]:
test_talk_word = []

for talk in test_data:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    test_talk_word.append(temp)
    
print(len(test_talk_word[0]))

In [ ]:
test_talk_word[0][:10]

In [ ]:
train_data[0][:5]

In [96]:
vocab = {}
N = 1

for talk in train_data:
    for sent in talk:
        for word in sent:
            if word in vocab:
                N = N
            else:
                vocab[word] = N
                N += 1
                
vocab["unknown_word"] = 0

In [97]:
vocab["unknown_word"]

0

In [ ]:
train_talk_word_index = []

for talk in train_talk_word:
    temp = []
    for word in talk:
        if word in vocab:
            temp.append(vocab[word])
        else:
            temp.append(vocab["unknown_word"])
    train_talk_word_index.append(temp)

In [ ]:
test_talk_word_index = []

for talk in test_talk_word:
    temp = []
    for word in talk:
        if word in vocab:
            temp.append(vocab[word])
        else:
            temp.append(vocab["unknown_word"])
    test_talk_word_index.append(temp)

In [ ]:
train_talk_word_index[0][:20]

In [ ]:
train_talk_word_index_exp = []

for i in range(len(train_talk_word_index)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (train_lenghts[i]-1):
            temp.append(train_talk_word_index[i][j])
        else:
            temp.append(0)
    train_talk_word_index_exp.append(temp)

In [ ]:
test_talk_word_index_exp = []

for i in range(len(test_talk_word_index)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (test_lenghts[i]-1):
            temp.append(test_talk_word_index[i][j])
        else:
            temp.append(0)
    test_talk_word_index_exp.append(temp)

In [ ]:
np.shape(test_talk_word_index_exp)

In [ ]:
np.shape(train_talk_word_index_exp)

In [ ]:
test_talk_word_index_exp[0]

In [58]:
train_labels_RNN = np.argmax(train_labels_onehot, 1) 

In [59]:
train_labels_RNN[:10]

array([0, 0, 0, 1, 5, 0, 0, 0, 0, 5])

In [62]:
test_labels_RNN = np.argmax(test_labels_onehot, 1) 

In [69]:
class SimpleDataIterator():
    def __init__(self, texts, lengths, labels):
        self.texts = texts
        self.lengths = lengths
        self.labels = labels
        self.size = len(self.texts)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        temp = list(zip(self.texts, self.lengths, self.labels))
        random.shuffle(temp)
        self.texts, self.lengths, self.labels = zip(*temp)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res_texts = self.texts[self.cursor:self.cursor+n]
        res_lengths = self.lengths[self.cursor:self.cursor+n]
        res_labels = self.labels[self.cursor:self.cursor+n]
        self.cursor += n
        return res_texts, res_labels, res_lengths

In [71]:
data = SimpleDataIterator(train_talk_word_index_exp, train_lenghts, train_labels_RNN)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')
print('Target values\n', d[1], end='\n\n')
print('Sequence lengths\n', d[2])

Input sequences
 ([815, 4711, 204, 3711, 446, 2367, 26061, 1245, 742, 2253, 1475, 3759, 13698, 3743, 1853, 636, 68, 2154, 30598, 27479, 1655, 632, 1362, 6615, 40, 1475, 3759, 742, 1362, 1323, 10927, 4914, 5445, 20098, 1655, 1899, 12886, 31526, 13820, 12674, 43840, 4947, 14495, 11695, 3872, 955, 284, 20979, 4186, 24664, 90, 284, 2675, 903, 21, 1878, 53, 6446, 18968, 829, 2244, 31237, 1461, 20536, 15572, 5442, 489, 2584, 316, 1597, 1598, 1790, 5422, 410, 10758, 4531, 2584, 432, 1708, 1121, 1012, 27100, 11678, 117, 7625, 3275, 438, 10513, 4422, 5505, 4102, 2718, 1236, 3188, 29887, 20337, 6551, 1576, 5051, 6788, 1769, 4643, 1769, 4705, 1688, 6788, 1701, 4731, 1078, 11047, 1595, 25146, 14192, 1062, 363, 48, 47, 436, 3223, 472, 1440, 1576, 2218, 523, 7625, 1040, 3125, 3125, 158, 1701, 1615, 458, 2374, 43841, 1063, 23548, 14192, 5211, 3711, 3125, 551, 1790, 12168, 3559, 11047, 3125, 2154, 3125, 23393, 12168, 1944, 10645, 7679, 3125, 18917, 12512, 1475, 3759, 1769, 12168, 1871, 1615, 1266, 762

In [81]:
np.shape(train_talk_word_index_exp)

(1835, 2221)

In [83]:
np.shape(test_talk_word_index_exp)

(250, 2221)

In [85]:
np.shape(train_lenghts)

(1835,)

In [86]:
np.shape(test_lenghts)

(250,)

In [87]:
np.shape(train_labels_RNN)

(1835,)

In [88]:
np.shape(test_labels_RNN)

(250,)

In [89]:
len(vocab)

51593

In [153]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = len(vocab),
    state_size = 100,
    batch_size = 50,
    num_classes = 8):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.constant(1.0)

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    cell = tf.nn.rnn_cell.LSTMCell(state_size)
    init_state = tf.get_variable('init_state', [1, 2*state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                 initial_state=init_state, dtype=tf.float32)

    # Add dropout, as the model otherwise quickly overfits
    #rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    """
    Obtain the last relevant output. The best approach in the future will be to use:

        last_rnn_output = tf.gather_nd(rnn_outputs, tf.pack([tf.range(batch_size), seqlen-1], axis=1))

    which is the Tensorflow equivalent of numpy's rnn_outputs[range(30), seqlen-1, :], but the
    gradient for this op has not been implemented as of this writing.

    The below solution works, but throws a UserWarning re: the gradient.
    """
    #idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    #last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)
    
    last_rnn_output = tf.reduce_mean(rnn_outputs, 1)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }

def train_graph(graph, batch_size = 50, num_epochs = 10, iterator = SimpleDataIterator):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        tr = iterator(train_talk_word_index_exp, train_lenghts, train_labels_RNN)
        te = iterator(test_talk_word_index_exp, test_lenghts, test_labels_RNN)

        step, accuracy = 0, 0
        tr_losses, te_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                te_epoch = te.epochs
                while te.epochs == te_epoch:
                    step += 1
                    batch = te.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                te_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- te:", te_losses[-1])

    return tr_losses, te_losses

In [154]:
g = build_graph(vocab_size=51594, batch_size=50, num_classes=8, state_size=100)

In [155]:
tr_losses, te_losses = train_graph(g, batch_size = 50, num_epochs = 10)

Accuracy after epoch 1  - tr: 0.565405406461 - te: 0.289999999106
Accuracy after epoch 2  - tr: 0.569999998642 - te: 0.288000002503


KeyboardInterrupt: 

RNNs have an advantage because for the words are inputted in a sequence there for there is a possiblity that the NN can actually infer something from the sequence rather than just the bag of words representation which completelly ignores order and context

Using only the last output offers no significant improvement.b